In [ ]:
# Импорт необходимых библиотек
import json
import boto3
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from dotenv import load_dotenv
from textblob import TextBlob
from decimal import Decimal

# Загрузка переменных окружения из .env файла
load_dotenv()

# Инициализация клиента STS и DynamoDB
sts_client = boto3.client('sts', region_name=os.getenv('AWS_REGION'))

# Принятие роли через STS
assumed_role = sts_client.assume_role(
    RoleArn=os.getenv('ROLE_ARN'),
    RoleSessionName='LambdaNewsAnalyzerSession'
)

credentials = assumed_role['Credentials']

# Инициализация клиента DynamoDB с временными креденшиалами
dynamodb = boto3.resource(
    'dynamodb',
    region_name=os.getenv('AWS_REGION'),
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken']
)

# Получение данных из DynamoDB
table = dynamodb.Table(os.getenv('DYNAMODB_TABLE'))

def fetch_data():
    response = table.scan()
    items = response.get('Items', [])
    return items

# Подготовка данных для обучения
def prepare_data(items):
    # Преобразуем данные в numpy массив
    X = []
    y = []

    for item in items:
        # Текстовые данные (описание) и метки (настроение)
        description = item['description']
        sentiment = float(item['sentiment'])

        # Добавляем описание и настроение в списки
        X.append(description)
        y.append(sentiment)

    return np.array(X), np.array(y)

# Определение модели
class SentimentModel(nn.Module):
    def __init__(self, input_size):
        super(SentimentModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Обучение модели
def train_model(X_train, y_train, epochs=100):
    model = SentimentModel(input_size=X_train.shape[1])  # Укажите размер входа
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    losses = []

    # Обучение
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train.unsqueeze(1))  # Убедитесь, что y_train имеет нужную размерность
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

    return model, losses

# Визуализация графика потерь
def plot_losses(losses):
    plt.figure(figsize=(10, 5))
    plt.plot(losses, label='Loss')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

if __name__ == "__main__":
    # Получение данных
    items = fetch_data()
    X, y = prepare_data(items)

    # Преобразование текстов в векторы (это необходимо для обучения модели)
    # Здесь может быть использована более сложная обработка текста, например, TF-IDF или Word Embeddings
    # Это может быть сделано с помощью библиотеки sklearn или gensim
    # Пример простого кодирования (вам нужно будет использовать векторизацию текстов)
    
    # Например, простой способ векторизации (это нужно заменить на TF-IDF или аналог)
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    vectorizer = TfidfVectorizer()
    X_vectorized = vectorizer.fit_transform(X).toarray()

    # Разделение на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

    # Обучение модели
    model, losses = train_model(torch.FloatTensor(X_train), torch.FloatTensor(y_train))

    # Визуализация графика потерь
    plot_losses(losses)

    # Оценка модели
    model.eval()
    with torch.no_grad():
        predictions = model(torch.FloatTensor(X_test))
        # Здесь можно добавить код для оценки модели, например, расчет RMSE или R^2
